In [1574]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 

In [1575]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"
input_directory = Path(f'{main_path}TikTok/Input')  
print(input_directory)
max_columns = 7


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input


In [1576]:
for i in range(max_columns):
        col_path = [f"col_path_{i}" for i in range(max_columns)]
        col_path_values = [f"col_path_{i}_values" for i in range(1, max_columns)]
        col_path_list = [f"col_path_{i}_lIST" for i in range(1, max_columns)]

In [1577]:
print(col_path_values)

['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values', 'col_path_6_values']


### loading_data()

In [1578]:
def loading_data(data):
    

    with open(data, 'r') as file:
        data = json.load(file)

    df = pd.DataFrame()
    df['col_path_0_values'] = data
    
    df.reset_index(inplace = True)
    df = df.drop('index', axis = 1)

    for i in range(max_columns):
        df[f'col_path_{i}'] = ''

    for i in range(1,max_columns):   
        df[f'col_path_{i}_values'] = ''
        df[f'col_path_{i}_LIST'] = ''
    
    return(df)



### flatten_json()

In [1579]:
def flatten_json(df):
    
    #df['last_valid_index'] = ''

    for i in range(max_columns):
        for ix, row in df.iterrows():
            value= row[f'col_path_{i}_values']
            
            if isinstance(value, dict):
                
                for k, v in value.items():
                    if isinstance(v, list):
                        v = v[0]
                        list_holder = 'LIST'
                    
                    
                    else:
                            list_holder = 'NO LIST'
                    

                    new_row = row.copy()
                    new_row[ f'col_path_{i+1}_values'] = v
                    new_row[ f'col_path_{i}'] = k
                    new_row[f'col_path_{i+2}_LIST'] = list_holder
                    df.loc[len(df)] = new_row
                        


            else:
                continue

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df_red = df.drop(columns=[col for col in df.columns if col.endswith("_LIST")])
    df_red['last_valid_index'] = df_red.apply(pd.Series.last_valid_index, axis=1)
    df_red = df_red.drop(columns=[col for col in df.columns if col.endswith("values")])


    df = pd.merge(df, df_red, on = col_path, how='left')
    
    
    return(df)

### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [1580]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']



# Define the function 'process_col_path()'
def process_col_path(df, col_path_values, data_types):
    df['data_type'] = ''

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """
    
    for ix, row in df.iterrows():
        for col in col_path_values:
            

            #If the value stored in the column is found in the list 'data_types', 
            if row[col] in data_types:
                # this value is placed in the column 'data_type'
                value = row[col]
                
                df.at[ix, 'data_type'] = value
            else:
                continue
               
    
                
    return df

### remove_intermediate_rows()

In [1581]:
def remove_intermediate_rows(df):
    drop_index = []
    for ix, row in df.iterrows():
            last_value = row[f"{row['last_valid_index']}"]
            #print(last_value)
            #print(df.columns)
        
            if isinstance(last_value, str):
                    if last_value in (data_types):
                            last_value
                            
            else:
                   drop_index.append(ix)
    df = df.drop(drop_index)
    display(df)
    return df

### extract_path()

In [1582]:
def extract_path(df, max_columns):
    
    df['path']= ''
    
    for ix, row in df.iterrows():
        path = []
    
        for col in [f"col_path_{i}" for i in range(max_columns)]:
            val = row[col]
        
            if pd.notna(val):

                path.append(str(val).strip())

      
        #print(path)
        df.at[ix, 'path'] = path
    
        

    return df




### list_path()

In [1583]:
def list_path(df):
    
    df['list_path'] = ''
    df['subfield_path'] = ''
    df['column_name'] = ''
    df['var_type'] = ''
    
    for ix, row in df.iterrows():
        
        path = row['path'] 
        #print(path)
        
        #for i in range(1, len(path)):
        for i in range(1, max_columns):
            if row[f"col_path_{i}_LIST"] == "LIST":
                var_type = 'list'
                list_path = json.dumps(path[:-1])
                subfield_path = path[-1]
                column_name = subfield_path
                subfield_path = json.dumps(subfield_path)
            elif row[f"col_path_{i}_LIST"] == "NO LIST":
                var_type = 'static'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
            elif row[f"col_path_{i}_LIST"] == "MISSING" :
                var_type = 'skip'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
    
        df.at[ix, 'list_path'] = list_path
        df.at[ix, 'subfield_path'] = subfield_path
        df.at[ix, 'column_name'] = column_name
        df.at[ix, 'var_type'] = var_type
    
      
        
        return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [1584]:
def clean_and_store(df, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """

  
   
    df=  df[df['col_path_0'] != 'Direct Message']
    
    
  
    df = df[['column_name', 'path', 'list_path', 'subfield_path', 'var_type', 'data_type']]
    # Save the DataFrame 
    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [1585]:
def structure_donations(data, col_path, max_columns):
     # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    df = loading_data(data)
    print('FINISH: loading_data()')
    df = flatten_json(df)
    print('FINISH: flatten_json()')
    df = process_col_path(df, col_path_values, data_types)
    print('FINISH: process_col_path()')
    df = remove_intermediate_rows(df)
    print('FINISH: remove_intermediate_rows()')
    #display(df)
    df = extract_path(df, max_columns)
    print('FINISH: extract_path()')
    df = list_path(df)
    print('FINISH: list_path()')
    df = clean_and_store(df, file_name)
    print('FINISH: clean_and_store()')

    return(df)
    
    

    

## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [1586]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        print(file.name)
        structure_donations(file, col_path, max_columns)

HJ_tiktol_json_strcure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()


/tmp/ipykernel_989656/4017166184.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,col_path_0_values,col_path_0,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_6,col_path_1_values,col_path_1_LIST,...,col_path_3_values,col_path_3_LIST,col_path_4_values,col_path_4_LIST,col_path_5_values,col_path_5_LIST,col_path_6_values,col_path_6_LIST,last_valid_index,data_type
57,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Ad Interests,AdInterestCategories,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,string,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,string
58,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Instant Form Ads Responses,ResponsesList,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
59,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
60,{'Ads and data': {'Ad Interests': {'AdInterest...,App Settings,Block List,App,NaN,NaN,NaN,NaN,"{'Block List': {'App': 'number', 'BlockList': ...",NaN,...,number,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,number
61,{'Ads and data': {'Ad Interests': {'AdInterest...,App Settings,Block List,BlockList,NaN,NaN,NaN,NaN,"{'Block List': {'App': 'number', 'BlockList': ...",NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,{'Ads and data': {'Ad Interests': {'AdInterest...,Direct Message,Direct Messages,ChatHistory,Chat History with jannemertens99:,From,NaN,NaN,{'Direct Messages': {'ChatHistory': {'Chat His...,NaN,...,{'Chat History with graaciel_:': [{'Date': 'st...,NO LIST,"{'Date': 'string', 'From': 'string', 'Content'...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string
240,{'Ads and data': {'Ad Interests': {'AdInterest...,Direct Message,Direct Messages,ChatHistory,Chat History with jannemertens99:,Content,NaN,NaN,{'Direct Messages': {'ChatHistory': {'Chat His...,NaN,...,{'Chat History with graaciel_:': [{'Date': 'st...,NO LIST,"{'Date': 'string', 'From': 'string', 'Content'...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string
241,{'Ads and data': {'Ad Interests': {'AdInterest...,Direct Message,Direct Messages,ChatHistory,Chat History with nynkeveenstraa:,Date,NaN,NaN,{'Direct Messages': {'ChatHistory': {'Chat His...,NaN,...,{'Chat History with graaciel_:': [{'Date': 'st...,NO LIST,"{'Date': 'string', 'From': 'string', 'Content'...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string
242,{'Ads and data': {'Ad Interests': {'AdInterest...,Direct Message,Direct Messages,ChatHistory,Chat History with nynkeveenstraa:,From,NaN,NaN,{'Direct Messages': {'ChatHistory': {'Chat His...,NaN,...,{'Chat History with graaciel_:': [{'Date': 'st...,NO LIST,"{'Date': 'string', 'From': 'string', 'Content'...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string


/tmp/ipykernel_989656/4017166184.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
extra_tiktok_json_structure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()


...

FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_json_structure_MvdV.json
FINISH: loading_data()


/tmp/ipykernel_989656/4017166184.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: flatten_json()
FINISH: process_col_path()


,col_path_0_values,col_path_0,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_6,col_path_1_values,col_path_1_LIST,...,col_path_3_values,col_path_3_LIST,col_path_4_values,col_path_4_LIST,col_path_5_values,col_path_5_LIST,col_path_6_values,col_path_6_LIST,last_valid_index,data_type
57,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Ad Interests,AdInterestCategories,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,string,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,string
58,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Instant Form Ads Responses,ResponsesList,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
59,{'Ads and data': {'Ad Interests': {'AdInterest...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,NaN,NaN,NaN,NaN,{'Ad Interests': {'AdInterestCategories': 'str...,NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
60,{'Ads and data': {'Ad Interests': {'AdInterest...,App Settings,Block List,App,NaN,NaN,NaN,NaN,"{'Block List': {'App': 'number', 'BlockList': ...",NaN,...,number,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,number
61,{'Ads and data': {'Ad Interests': {'AdInterest...,App Settings,Block List,BlockList,NaN,NaN,NaN,NaN,"{'Block List': {'App': 'number', 'BlockList': ...",NaN,...,object,NO LIST,NaN,NO LIST,NaN,NaN,NaN,NaN,col_path_3_values,object
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,{'Ads and data': {'Ad Interests': {'AdInterest...,App Settings,Settings,SettingsMap,Push Notification,New Likes on My Video,NaN,NaN,"{'Block List': {'App': 'number', 'BlockList': ...",NaN,...,"{'Allow DownLoad': 'string', 'Allow Others to ...",NO LIST,"{'Desktop notification': 'string', 'New Commen...",NO LIST,string,NO LIST,NaN,NO LIST,col_path_5_values,string
192,{'Ads and data': {'Ad Interests': {'AdInterest...,Profile,Profile Info,ProfileMap,PlatformInfo,Description,NaN,NaN,"{'AI-Moji': {'CreateDate': 'string', 'AIMojiLi...",NaN,...,"{'PlatformInfo': [{'Description': 'string', 'N...",NO LIST,"{'Description': 'string', 'Name': 'string', 'P...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string
193,{'Ads and data': {'Ad Interests': {'AdInterest...,Profile,Profile Info,ProfileMap,PlatformInfo,Name,NaN,NaN,"{'AI-Moji': {'CreateDate': 'string', 'AIMojiLi...",NaN,...,"{'PlatformInfo': [{'Description': 'string', 'N...",NO LIST,"{'Description': 'string', 'Name': 'string', 'P...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string
194,{'Ads and data': {'Ad Interests': {'AdInterest...,Profile,Profile Info,ProfileMap,PlatformInfo,Platform,NaN,NaN,"{'AI-Moji': {'CreateDate': 'string', 'AIMojiLi...",NaN,...,"{'PlatformInfo': [{'Description': 'string', 'N...",NO LIST,"{'Description': 'string', 'Name': 'string', 'P...",NO LIST,string,LIST,NaN,NO LIST,col_path_5_values,string


FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()


## Merge all data structures into one schema_df

In [1587]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()


### Id creation

In [1588]:
id_df = merged_df[['path', 'column_name']]
id_df = id_df.drop_duplicates()
id_df[ 'id'] = ''



for n in range(1, max_columns):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = list(row['path'])
        path_zero = path[0]
        path_rest = path[-n:]  # Varies each loop to attempt uniqueness

        id_list = [path_zero] + path_rest
        new_id = ':'.join(id_list)

        id_df.at[i, 'id'] = new_id  # Properly update the DataFrame



merged_df = pd.merge(merged_df, id_df, on = ['path', 'column_name'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 
#merged_df['id']= merged_df['id'].fillna(merged_df['variable'])


#### ID duplicate flags

In [1589]:
merged_df = merged_df.drop_duplicates()
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')

/tmp/ipykernel_989656/29815283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup['duplicate_flag'] = 'Yes'


### Clean and Save

In [1590]:
merged_df['json_name'] = ''

keep_columns = ['id', 'column_name', 
                 'path', 'list_path', 'subfield_path', 
                 'var_type', 'data_type', 'json_name', 'duplicate_flag']


merged_df = merged_df[keep_columns ]

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)

In [1591]:
"""
max_columns = 7

with open(main_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame()
df['col_path_0_values'] = data
df.reset_index(inplace = True)
df = df.drop('index', axis = 1)

for i in range(max_columns):
    df[f'col_path_{i}'] = ''
    df[f'col_path_{i+1}_values'] = ''
    df[f'col_path_{i+1}_LIST'] = ''
    col_path = [f"col_path_{i}" for i in range(max_columns)]
    col_path_values = [f"col_path_{i+1}_values" for i in range(max_columns)]
    col_path_values = [f"col_path_{i+1}_lIST" for i in range(max_columns)]

   
for i in range(max_columns):
    for ix, row in df.iterrows():
        value= row[f'column_{i}_values']
        
        if isinstance(value, dict):
            
            for k, v in value.items():
                if isinstance(v, list):
                    v = v[0]
                    list_holder = 'LIST'
                
                
                else:
                        list_holder = 'NO LIST'
                

                new_row = row.copy()
                new_row[ f'column_{i+1}_values'] = v
                new_row[ f'column_{i}'] = k
                new_row[f'column_{i+2}_LIST'] = list_holder
                df.loc[len(df)] = new_row
                    


        else:
            continue



df = df.replace(r'^\s*$', np.nan, regex=True)
df['last_valid_index'] = df.apply(pd.Series.last_valid_index, axis=1)


data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']

for ix, row in df.iterrows():
        last_value = row[f"{row['last_valid_index']}"]

        if isinstance(last_value, str):
                if last_value in (data_types):
                        last_value
        else:
                df = df.drop(ix)
                
"""
        

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_989656/3928669051.py:1: SyntaxWarning: invalid escape sequence '\s'
  """


'\nmax_columns = 7\n\nwith open(main_path, \'r\') as file:\n    data = json.load(file)\n\ndf = pd.DataFrame()\ndf[\'col_path_0_values\'] = data\ndf.reset_index(inplace = True)\ndf = df.drop(\'index\', axis = 1)\n\nfor i in range(max_columns):\n    df[f\'col_path_{i}\'] = \'\'\n    df[f\'col_path_{i+1}_values\'] = \'\'\n    df[f\'col_path_{i+1}_LIST\'] = \'\'\n    col_path = [f"col_path_{i}" for i in range(max_columns)]\n    col_path_values = [f"col_path_{i+1}_values" for i in range(max_columns)]\n    col_path_values = [f"col_path_{i+1}_lIST" for i in range(max_columns)]\n\n\nfor i in range(max_columns):\n    for ix, row in df.iterrows():\n        value= row[f\'column_{i}_values\']\n\n        if isinstance(value, dict):\n\n            for k, v in value.items():\n                if isinstance(v, list):\n                    v = v[0]\n                    list_holder = \'LIST\'\n\n\n                else:\n                        list_holder = \'NO LIST\'\n\n\n                new_row = row.c